<a href="https://colab.research.google.com/github/Taka0007/2023seminar.2nd-term/blob/main/speech%20analysis%20/code/Noninterative_Reconstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 非反復の位相復元の実装
下記論文の実装　<br>
https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=7890450

In [4]:
import heapq
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.decomposition import NMF
from scipy import signal
from tqdm import tqdm
from IPython.display import Audio

def calculate_phase_gradient(S, a, b, lambda_L, M):
    """
    Calculate the phase gradient for STFT.

    :param S: STFT magnitude matrix.
    :param a, b: Hop sizes.
    :param lambda_L: Scaling parameter.
    :param M: Number of frequency bins.
    :return: Phase gradient matrix.
    """
    slog = np.log(S)
    Dt = np.diff(slog, axis=0, append=slog[-1:])
    Dω = np.diff(slog, axis=1, append=slog[:, -1:])

    φSC_ω = -lambda_L / (a * M) * Dt
    φSC_t = (a * M) / lambda_L * Dω + 2 * np.pi * a / M

    return np.stack([φSC_ω, φSC_t], axis=-1)

def integrate_phase_gradient(phase_gradient, tol=0.1):
    """
    Integrate the phase gradient using heap for path management.

    :param phase_gradient: Computed phase gradient.
    :param tol: Relative magnitude tolerance.
    :return: Reconstructed phase matrix.
    """
    magnitude = np.abs(phase_gradient)
    phase = np.zeros_like(magnitude)

    # Create a heap for managing integration paths
    heap = [(magnitude[i, j], i, j) for i in range(magnitude.shape[0]) for j in range(magnitude.shape[1])]
    heapq.heapify(heap)

    while heap:
        mag, i, j = heapq.heappop(heap)

        # Skip coefficients below tolerance
        if mag < tol:
            continue

        # Integrate phase gradient
        # This is a simplified version; in practice, you need to integrate along the paths efficiently.
        if i > 0:
            phase[i, j] += phase[i - 1, j] + phase_gradient[i - 1, j, 0]
        if j > 0:
            phase[i, j] += phase[i, j - 1] + phase_gradient[i, j - 1, 1]

    return phase


# Load audio file
file_path = '/content/Vinyl-sabi.wav'
audio_data, sr = librosa.load(file_path)

# ガウス窓の標準偏差（この値は実験的に決定する）
std = 2048 / 6

# Compute the STFT (STFT magnitude matrix)
window = signal.windows.gaussian(2048, std)
stft_matrix = librosa.stft(audio_data, n_fft=2048, hop_length=80, win_length=2048, window=window)

a, b = 10, 10
lambda_L = 1
M = 100

phase_gradient = calculate_phase_gradient(stft_matrix, a, b, lambda_L, M)

# STFTの振幅成分を取得
magnitude = np.abs(stft_matrix)

# 推定された位相の結果を組み合わせる
combined_stft = magnitude * np.exp(1j * phase_gradient)

# 2. 逆STFTの実行
reconstructed_signal = librosa.istft(combined_stft, hop_length=80, win_length=2048, window='gaussian')

# 3. 音声の再生
Audio(reconstructed_signal, rate=sr)

<ipython-input-4-cb47cad6363a>:66: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:183: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)
<ipython-input-4-cb47cad6363a>:85: RuntimeWarning: overflow encountered in exp
  combined_stft = magnitude * np.exp(1j * phase_gradient)


ValueError: ignored